# Finding data

## Using the web interface

You can click down the hierachy or use the find tool

## Using flexiznam

In [ ]:
# import and get a token to log in to flexilims
import flexiznam as flz

flm_sess = flz.get_flexilims_session(project_id="demo_project")

In [ ]:
mouse = flz.get_entity(name="PZAJ5.1a", datatype="mouse", flexilims_session=flm_sess)
print(mouse)

In [ ]:
sess = flz.get_children(
    parent_id=mouse.id, children_datatype="session", flexilims_session=flm_sess
)
sess

In [ ]:
sess = sess.loc[0]
sess

In [ ]:
datasets = flz.get_entities(datatype="dataset", flexilims_session=flm_sess)
datasets

See also:
- `flz.get_experimental_session`
- `flz.get_datasets`

# Analysing data

## Find path and load data

Once you have some entity out of the database you will want to access the data. The path stored in the database is relative

In [ ]:
overlay = flz.get_entity(
    name="PZAJ5.1a_brain_slide4_slice3_stack_30X_overlay_example.png",
    datatype="dataset",
    flexilims_session=flm_sess,
)
overlay.path

You can create the full path using config info and the `is_raw` flag

In [ ]:
from pathlib import Path

data_root = flz.PARAMETERS["data_root"]
path = Path(data_root["raw"] if overlay.is_raw else data_root["raw"]) / overlay.path
print(path)
print(path.is_file())

## Do analysis

### Create container

In [ ]:
from flexiznam.schema import Dataset

conflicts = "append"
exp_session = flz.get_entity(
    datatype="session", name="PZAJ5.1a_S20210513", flexilims_session=flm_sess
)

suite2p_dataset = Dataset.from_origin(
    project="demo_project",
    origin_type="session",
    origin_id=exp_session["id"],
    dataset_type="suite2p_rois",
    conflicts=conflicts,
    flexilims_session=flm_sess,
)

### Update it

In [ ]:
print(suite2p_dataset.flexilims_status())
print(suite2p_dataset.is_raw)

In [ ]:
# do analysis
suite2p_dataset.extra_attributes = dict(param1=3, param2="deconvolve")

### Upload it

In [ ]:
suite2p_dataset.update_flexilims()

In [ ]:
suite2p_dataset.flexilims_status()

In [ ]:
suite2p_dataset.extra_attributes["param2"] = "convolve"
suite2p_dataset.flexilims_status()

In [ ]:
suite2p_dataset.flexilims_report()

In [ ]:
suite2p_dataset.update_flexilims()

## Repeat

In [ ]:
suite2p_dataset = Dataset.from_origin(
    project="demo_project",
    origin_type="session",
    origin_id=exp_session["id"],
    dataset_type="suite2p_rois",
    conflicts=conflicts,
    flexilims_session=flm_sess,
)
suite2p_dataset.extra_attributes = dict(param2="anothermethod")
suite2p_dataset.update_flexilims()

## Analyse the next step

In [ ]:
datasets = flz.get_entities(
    origin_id=exp_session.id, datatype="dataset", flexilims_session=flm_sess
)
datasets[(datasets.dataset_type == "suite2p_rois")]

In [ ]:
datasets[(datasets.dataset_type == "suite2p_rois") & (datasets.param2 == "deconvolve")]